Predicting Titanic Survival Using Logistic Regression

Dataset: Titanic Dataset

Preprocessing Steps:
Handle missing values (e.g., fill missing ages with median).
Encode categorical variables (e.g., one-hot encoding for embarked and gender).
Standardize numerical features.

Task:
Implement logistic regression to predict survival on the Titanic and evaluate the model using ROC-AUC.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
data = pd.read_csv('/content/titanic.csv.csv') #load data
print(data.head())#data visibility

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
data = data.isnull().sum()

In [ ]:
print(data)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [ ]:
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
data['Cabin'].fillna('Unknown', inplace=True)

In [32]:
#encoding categorical values
x = data.drop('Survived', axis = 1)
y = data['Survived']

categorical_values = ['Sex', 'Embarked', 'Cabin', 'Name']
numerical_values = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch']

preprocesser = ColumnTransformer(
    transformers = [
        ('categs',  OneHotEncoder(handle_unknown='ignore'), categorical_values),
        ('num', StandardScaler(), numerical_values)
    ]
)

In [33]:
print(preprocesser)

ColumnTransformer(transformers=[('categs',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sex', 'Embarked', 'Cabin', 'Name']),
                                ('num', StandardScaler(),
                                 ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch'])])


In [34]:
#splitting data set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [35]:
model = Pipeline([
    ('preprocesser', preprocesser),
    ('model', LogisticRegression())
])
model.fit(x_train, y_train)

Pipeline(steps=[('preprocesser',
                 ColumnTransformer(transformers=[('categs',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked', 'Cabin',
                                                   'Name']),
                                                 ('num', StandardScaler(),
                                                  ['Age', 'Fare', 'Pclass',
                                                   'SibSp', 'Parch'])])),
                ('model', LogisticRegression())])

In [36]:
# Make predictions
y_pred_proba = model.predict_proba(x_test)[:, 1]

In [37]:
# Evaluate the model using ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.8806949806949808
